# Pandas and duckdb

### Task 0 - Load the sakila sqlite database into a duckdb database file

In [ ]:
import duckdb 

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC").df()

description

## Read all data into dictionary of pandas dataframes

In [ ]:
dfs = {}

with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        dfs[name] = conn.sql(f"FROM staging.{name};").df()
        duckdb.register(name, dfs[name])

dfs.keys()

## Task 1 - EDA in python

### Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [ ]:
long_movies = duckdb.sql("""
SELECT title, length AS duration
FROM film
WHERE length > 180; 
""").df()

long_movies.head()

### Which movies have the word "love" in its title? Show the following columns
- title
- rating
- length
- description

In [ ]:
love_word = duckdb.sql("""
SELECT title, rating, length, description 
FROM film
WHERE title ILIKE '%love%'; 
""").df()

love_word.head()

### Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [ ]:
aggregate_function = duckdb.sql("""
SELECT 
    MIN(length) AS min_length_movies, 
    ROUND(AVG(length)) AS mean_length_movies, 
    MEDIAN(length) AS median_length_movies, 
    MAX(length) AS max_length_movies
FROM film
""").df()


aggregate_function

### The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie. The Manager wants to know the 10 most expensive movies to rent per day.

In [ ]:
rental_rate = duckdb.sql("""
SELECT title, rental_rate, rental_duration, (rental_rate / rental_duration) AS cost_per_day                
FROM film
ORDER BY cost_per_day DESC;
""").df()

rental_rate.head(10)

###  Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [ ]:
top_10_actors = duckdb.sql("""
SELECT a.actor_id, a.first_name, a.last_name, COUNT(*) AS film_count                
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.actor_id, a.first_name, a.last_name
ORDER BY film_count DESC;
""").df()

top_10_actors

### Now it's time for you to choose your own question to explore the sakila database! Write down 3-5 questions you want to answer and then answer them using pandas and duckdb.

#### Which films are rented most? 

In [ ]:
most_rented_films = duckdb.sql("""
SELECT 
    f.title,                                                                              
    COUNT(r.rental_id) AS total_rentals,
FROM film f
JOIN inventory i ON f.film_id = i.film_id                               
JOIN rental r ON i.inventory_id = r.inventory_id                               
GROUP BY f.title
ORDER BY total_rentals DESC                                                                                                                     
LIMIT 20;                                                                                                                                                                                                       
""")

most_rented_films

### How do stores compare in revenue and number of rentals?

In [ ]:
store_revenue = duckdb.sql("""
SELECT
    s.store_id,                           
    COUNT(r.rental_id) AS total_rentals,                           
    ROUND(SUM(p.amount)) AS total_revenue_in_usd
FROM store s
JOIN staff st ON s.store_id = st.store_id                                                                                 
JOIN payment p ON st.staff_id = p.staff_id                                                                                 
JOIN rental r ON p.rental_id = r.rental_id
GROUP BY s.store_id;                                                                                                           
""")

store_revenue

#### Rentals based on the yearly seasons

In [ ]:
season_rentals = duckdb.sql("""
SELECT 
    CASE 
        WHEN STRFTIME('%m', rental_date) IN ('12','01','02') THEN 'Winter'                                                        
        WHEN STRFTIME('%m', rental_date) IN ('03','04','05') THEN 'Spring'                                                        
        WHEN STRFTIME('%m', rental_date) IN ('06','07','08') THEN 'Summer'                                                        
        WHEN STRFTIME('%m', rental_date) IN ('09','10','11') THEN 'Autumn' 
    END AS season,
    COUNT(*) AS rentals
FROM rental
GROUP BY season
ORDER BY rentals DESC;                                                                                                                                                                                       
""")

season_rentals

# Task 2 - Graphs

### Who are our top 5 customers by total spend? The Manager wants to know so that they can reward them with special offers. Create a bar chart showing the top 5 customers by total spend.

In [ ]:
top_5_customers = duckdb.sql("""
SELECT c.customer_id, 
        c.first_name || ' ' || c.last_name AS customer_name, 
        SUM(p.amount) AS total_spend
FROM customer c 
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id, customer_name
ORDER BY total_spend DESC;                            
""").df()

top_5_customers.head()

In [ ]:
ax = top_5_customers.head().plot(
    kind="barh",
    x="customer_name",
    y="total_spend",
    title="Top 5 customers by totalt spend",
    xlabel="Total spending in ($)",
    ylabel="Customer name"
)
ax.invert_yaxis()

### How much money does each film category bring in? Make a bar chart showing total revenue per film category.

In [ ]:
revenue_per_category = duckdb.sql("""
SELECT c.name AS category, 
        SUM(p.amount) AS total_revenue
FROM category c
JOIN film_category fc ON c.category_id = fc.category_id
JOIN film f ON fc.film_id = f.film_id
JOIN inventory i ON f.film_id = i.film_id                                                                                                                                                                            
JOIN rental r ON i.inventory_id = r.inventory_id                                                                                                                                                                            
JOIN payment p ON r.rental_id = p.rental_id
GROUP BY c.name
ORDER BY total_revenue DESC;                                                                                                                                                                                                                                                
""").df()

revenue_per_category

In [ ]:
ax = revenue_per_category.plot(
    kind="bar",
    x="category",
    y="total_revenue",
    title="Total revenue per film category",
    xlabel="Category",
    ylabel="Total revenue ($)"
)

ax.invert_xaxis()